# Сравнение качества моделей
### Содержание

* [Подготовка данных](#1)
* [Сравнение моделей](#2)
    * [Теоретические основы](#21)


## Подготовка данных <a id="1"></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("cleaned_dataset.csv", index_col=0)

Вначале нужно определить, на каких данных будет обучаться модель, а на каких тестироваться. **Традиционный подход** - это разделение исходного набора данных на 3 части (обучение, валидация и тестирование) с пропорции 60/20/20. В данном случае обучающая выборка используется для обучения модели, а валидация и тестирование для получения значения метрики без эффекта переобучения.

Однако существует и другой подход к разбиению данных - разделение на 2 части (обучение и тестирование) по правилу 80-20 (80% тренировочный, 20% тестовый). Зачастую данный метод применяется в тех случаях, когда отсутствует достаточное количество данных как в обучающем, так и в проверочном наборе.  

Перед тем как начать разбивать данные необходимо выделить из исходного набора данных целевую переменную (столбец `DRK_YN`) и сохранить её в отдельную переменную. Ниже приведён код разделения:

In [2]:
x = df.drop(columns=["DRK_YN"], axis=1)
y = df["DRK_YN"]

В ходе выполнения данной работы будет использован подход с разделением исходной выборки на 2 части с пропорцией 80-20, поскольку данный способ является самым популярным способом разбиения данных. Для того, чтобы разбить данные таким образом, существует специальный метод `train_test_split` в библиотеке `scikit-learn`.

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

## Сравнение моделей <a id="2"></a>

### Теоретические основы <a id="21"></a>

После того, как модели были обучены, их необходимо оценить при помощи имеющихся метрик оценки. Всего для задач классификации существует множество оценок. Но все они строятся на основе **матрицы ошибок** (*Confusion Matrix*).

Confusion matrix состоит из 4 основных исходов бинарной классификации: *истинно положительных* (**TP**), *ложно положительных* (**FP**), *истнно отрицательных* (**TN**) и *ложно отрицательных* (**FN**). 

Пусть какой-то набор медицинских данных характерен для данного диагноза. Если наша модель верно определила и поставила положительный класс, тогда это *истинно положительный* исход, если же модель ставит отрицательную метку класса, тогда это *ложно отрицательный* исход. В случае отсутствия диагноза у рассматриваемого набора данных исходы модели остаются аналогичными. Тогда, если модель относит запись к классу положительному, то мы говорим о *ложно положительном* исходе (здоровому человеку ошибочно поставлен диагноз "болен"), и наоборот, если модель определяет запись как отрицательный класс, то от *истинно отрицательный* исход (больному человеку поставлен диагноз "болен").

Одной из наиболее простых метрик, построенных на основе матрицы ошибок является **полнота** (*Recall*). Полнота определяет долю верных прогнозов среди всех данных с положительным классом. Она определяется отношением истинно положительных результатов к общему количеству положительных результатов. Таким образом формула выглядит следующим образом:
$$ Recall = \frac{TP}{TP+FN} $$
В исходной задачи необходимо уделить данной задаче некоторое внимание, поскольку в задаче ошибка нераспознания положительного класса высока.

Однако одна полнота не способна в полной мере оценить точность классификации. Поэтому вместе с ней применяют и другую метрику - **точность** (*Precision*), которая измеряет, насколько хорошо модель справляется с ложно положительными результатами. Более формально, точность принимает в расчет только те точки данных, которые были помечены как положительные, и определяет, сколько среди них истинно положительные. Формула для подсчета данного значения выглядит следующим образом:
$$Precision = \frac{TP}{TP+FP}$$
Данная оценка также является важной, поскольку в рамках данной задачи "цена" ложно положительного класса также высока.

Далее, поскольку ранее было выяснено, что метрики **полнота** и **точность** имеют равную высокую значимость, то лучше всего использовать **F-меру** (*F-score*), которая является средним гармоническим между первой и второй метриками. Почему при этом используется именно среднее гармоническое, а не среднее арифметическое? Потому что если одна метрика очень большая, например 100%, а другая наоборот крайне низкая (50%), то среднее арифметическое будет равняться 75%, что является приемлемым результатом. Однако в реальность такой классификатор имеет крайне низкую точность классификации. Таким образом, формула для подсчета выглядит так:
$$F_1 = \frac{2 \cdot precision \cdot recall}{precision+recall}$$
Если обе метрики большие, то и итоговый результат также высокий. Цель F1-меры состоит в том, чтобы определить, является ли высокой как полнота, так и точность.

Однако иногда требуется, чтобы либо полнота, либо точность превалировала одна над другой. Таким образом, может потребоваться при вычислении F-меры придать одной из них больший вес. Как раз для этого и была придумана $F_\beta$-мера, формула которой представлена ниже:
$$F_\beta = \frac{(1+\beta^2) \cdot precision \cdot recall}{\beta^2 \cdot precision + recall}$$
В данной формуле параметр $\beta$ может принимать любое положительное значение и обозначает уровень значимости одного из критериев. Чем ниже $\beta$, тем более весомой становится точность, а чем выше $\beta$, тем важнее становится полнота.

Из ранее рассмотренных метрик становится очевидно, что существует два типа ошибок: ложно положительные и ложно отрицательные. И существует метрика, которая основана на двух данных ошибках - **кривая рабочей характеристики приемника** (*ROC*). Данная кривая строится на основе двух показателей: **избирательности** (*False Positive Rate (FPR)*) и **полноте** (*True Positive Rate (TPR)*). Данные показатели высчитываются по следующим формулам:
$$ FPR = \frac{FP}{FP + TN} $$
$$ TPR = \frac{TP}{TP + TN} $$
И далее, на основе данных двух метрик строится ROC-кривая, где по горизонтали откладывается FPR, а по вертикали - TPR. Однако сама по себе ROC-кривая может лишь графически показать, какая модель лучше справляется с классификацией. Для того, чтобы сравнить две модели численно, необходимо найти **площадь под ROC-кривой** (*Area Under Curve ROC (AUC ROC)*). Диапазон значений AUC ROC равен $[0.5, 1]$, где чем ближе к 1, тем лучше модель для классификации данных.

### Практическая реализация <a id="22"></a>